# Цель работы.

**Цель работы**:
Анализ базы данных сервиса покупки книг по подписке. 

**Задачи**:
- Изучить состав базы данных (количество и состав таблиц), 
- Выполнить запросы, которые помогут сформулировать ценностное предложение для нового продукта.

# 1. Подключение к БД и загрузка таблиц
Для начала необходимо подключиться к базе данных и загрузить имеющиеся таблицы. Для загрузки таблиц использую функцию `table_loading`, которая поможет загрузить таблицы БД.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student',
             'pwd': 'Sdf4$2;d-d30pp',
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, 
             'db': 'data-analyst-final-project-db'} 

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
def table_loading(table):
    
    query = '''
    select *
    from %s        
    ''' % table
    data = pd.io.sql.read_sql(query, con = engine)
    
    print('Количество строк таблицы: {}'.format(len(data)))
    print('*'*50)
    display(data.head())

# Первичный анализ таблиц
Загрузим таблицы и изучим их содержимое: 

### Таблица `books` - "книги" 
- В таблице `books` - 1000 строк, 
- Primary Key - `book_id`, 
- Foreign Key - `author_id`, `publisher_id`	

In [4]:
table_loading('books')

Количество строк таблицы: 1000
**************************************************


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Таблица `authors` - "авторы" 
- В таблице `authors` - 636 строк, 
- Primary Key - `author_id`, 
- Foreign Key - отсутсвует.	

In [5]:
table_loading('authors')

Количество строк таблицы: 636
**************************************************


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Таблица `publishers` - "издатели" 
- В таблице `publishers` - 340 строк, 
- Primary Key - `publisher_id`, 
- Foreign Key - отсутсвует.	

In [6]:
table_loading('publishers')

Количество строк таблицы: 340
**************************************************


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Таблица `ratings` - "рейтинг" 
- В таблице `ratings` - 6456 строк, 
- Primary Key - `rating_id`, 
- Foreign Key - `book_id`.	

In [7]:
table_loading('ratings')

Количество строк таблицы: 6456
**************************************************


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Таблица `reviews` - "обзоры" 
- В таблице `reviews` - 2793 строки, 
- Primary Key - `review_id`, 
- Foreign Key - `book_id`.	

In [8]:
table_loading('reviews')

Количество строк таблицы: 2793
**************************************************


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Выполнение запросов
Для отображения результатов запросов использую функцию `request`, аргумент которой- запрос к БД. 

In [9]:
def request(query):
    request_result = pd.io.sql.read_sql(query, con = engine)
    return request_result

## Задача 1 - Посчитайте, сколько книг вышло после 1 января 2000 года.
Для того, чтобы решить задачу, выгрузим уникальные идентификаторы книг `book_id` и установим условие- дата выпуска книг должна быть позднее 01.01.2000 года. Результат - таких книг в БД **821 штука.**

In [10]:
query_1 = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date >= '2000-01-01'
'''

In [11]:
request(query_1)

,count
0,821


## Задача 2 - Для каждой книги посчитайте количество обзоров и среднюю оценку.

Для решения задачи:

1. Временная таблица №1 - выгружу из таблицы `reviews`, сгруппированной по колонке `book_id`, идентификаторы книг `book_id` и количество обзоров по каждой из книг COUNT(review_id) AS review_counter. Обзоры есть не на все книги в базе. 
2. Временная таблица №2 - из таблицы `ratings`, сгруппированной по колонке `book_id`, получим идентификаторы книг `book_id` и среднюю оценку каждой книги AVG(rating) as avg_rating.
3. При помощи LEFT JOIN соединим обе таблицы по колонке `book_id`. 

Таким образом, для каждой книги посчитан средний рейтинг и количество обзоров. 

In [12]:
query_2 = '''
WITH books_rev
AS
(SELECT book_id, COUNT(review_id) AS review_counter
FROM reviews
GROUP BY book_id),

book_rait AS
(SELECT book_id, AVG(rating) as avg_rating
FROM ratings
GROUP BY book_id)

select bra.book_id, avg_rating, review_counter
FROM book_rait AS bra
LEFT JOIN books_rev AS bre ON bra.book_id = bre.book_id
'''

In [13]:
request(query_2)

,book_id,avg_rating,review_counter
0,1,3.666667,2.0
1,2,2.500000,1.0
2,3,4.666667,3.0
3,4,4.500000,2.0
4,5,4.000000,4.0
...,...,...,...
995,996,3.666667,3.0
996,997,3.400000,3.0
997,998,3.200000,4.0
998,999,4.500000,2.0


## Задача 3 - Определите издательство, которое выпустило наибольшее число книг толще 50 страниц 

Для решения задачи: 
1. Выгружу из таблицы `books` идентификатор издателя `publisher_id` и количество книг `OUNT (book_id)` по каждому издательству, задам фильтр: количество страниц `num_pages` должно быть больше или равно 50, сделаю группировку по `publisher_id`. Результат запроса сохраню как временную таблицу `cte`.
2. Соединю таблицу publishers и временной таблицей `cte` по полю `publisher_id`, выгружу поля идентификатор издателя `publisher_id`, название издателя `publisher` и количество книг. Т.к. нам необходимо издательство, которое выпустило наибольшее количество книг, отсортируем результат запроса по убыванию количества выпущенных книг и ограничим выгрузку одним издательством с наибольним количеством книг. 

Победителем стал издатель `Penguin Books` (publisher_id = 242), выпустивший 42 книги. 

In [14]:
query_3 = '''
WITH cte
AS
(SELECT publisher_id, COUNT (book_id)
FROM books
WHERE num_pages >= 50
GROUP BY publisher_id)

SELECT p.publisher_id, publisher, count
FROM publishers AS p
JOIN cte ON p.publisher_id = cte.publisher_id
ORDER BY count DESC
LIMIT 1
'''

In [15]:
request(query_3)

,publisher_id,publisher,count
0,212,Penguin Books,42


## Задача 4 -  Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

Здесь я предлагаю 2 варианта решения задачи. 

**Вариант 1 - без использования оконных функций:**
1. Временная таблица №1 - `book_cte`. Из таблицы рейтингов `ratings`, сгруппированной по полю `book_id` выгрузим `book_id`, количество оценок `COUNT(rating_id) as counter` и среднюю оценку `AVG(rating) as book_rating`. После группировки применим фильтр - количество оценок `COUNT(rating_id)` должно быть более 50.
2. Временная таблица №2 - `final_cte`. Таблицу `book_cte` соединим с таблицей `books` по ключу `book_id` и с таблицей `authors` по ключу `author_id`. Сгруппируем получившуюся таблицу по колонке авторов `author_id`, выведем идентификатор автора `author_id` и средний рейтинг книг по автору `avg_rating`, найденный как среднее от средней оценки книг автора. Сделаем группировку рейтинга по убыванию `ORDER BY author_rating DESC` и выведем автора с самым большим рейтингом `LIMIT 1`. 
3. Полученный результат снова соединим с таблицей `authors`, чтобы получить не только идентификатор автора `author_id`, но и его ФИО. 

Авторoм с самым высоким рейтингом книг стала Джоан Роулинг (рейтинг 4.28)

In [16]:
query_4 = '''
WITH book_cte
AS
(SELECT book_id, COUNT(rating_id) as counter, AVG(rating) as book_rating
FROM ratings
GROUP BY book_id
HAVING COUNT(rating_id) > 50), 

final_cte
AS
(SELECT b.author_id, AVG(book_rating) as avg_rating
FROM book_cte AS bc
LEFT JOIN books AS b ON bc.book_id = b.book_id
LEFT JOIN authors AS au ON au.author_id = b.author_id
GROUP by b.author_id 
)

SELECT a.author_id, a.author, ROUND(fc.avg_rating, 2)
FROM authors AS a
JOIN final_cte AS fc ON fc.author_id = a.author_id
ORDER BY ROUND(fc.avg_rating, 2) DESC 
LIMIT 1
'''

In [17]:
request(query_4)

,author_id,author,round
0,236,J.K. Rowling/Mary GrandPré,4.28


**Вариант 2 - с использованием оконных функций:**

1. Временная таблица №1 - `book_rating`. Из таблицы рейтингов `ratings`, сгруппированной по полю `book_id` выгрузим `book_id`, количество оценок `COUNT(rating_id) as counter` и среднюю оценку `AVG(rating) as book_rating`. После группировки применим фильтр - количество оценок `COUNT(rating_id)` должно быть более 50.
2. Временная таблица №2 - `book_authors`. К таблице `books` присоединим таблицу авторов `authors` по ключу `author_id` и выгрузим идентификатор книги `book_id`, идентификатор `author_id` и ФИО `author` автора. 
3. Соединим `book_rating` и `book_authors` по ключу `book_id`. Здесь используем именно левое присоединение, т.к. нам нужны авторы только тех книг, которые есть в таблице `book_rating`. 
4. При помощи оконной функции `OVER (PARTITION BY author)` сделаем группировку по каждому автору и посчитаем средний рейтинг его книг `AVG(book_rating)`. 
5. Сделаем группировку рейтинга по убыванию `ORDER BY author_rating DESC` и выведем автора с самым большим рейтингом `LIMIT 1`. 

Авторoм с самым высоким рейтингом книг стала Джоан Роулинг (рейтинг 4.28)

In [18]:
query_4_1 = '''
WITH book_rating
AS
(SELECT r.book_id, COUNT(rating_id) as counter, AVG(rating) as book_rating
FROM ratings AS r
GROUP BY r.book_id
HAVING COUNT(rating_id) > 50), 

book_authors AS
(SELECT book_id, au.author_id, au.author
FROM books as b
JOIN authors as au ON b.author_id = au.author_id)

SELECT DISTINCT author, author_id, AVG(book_rating) OVER (PARTITION BY author) AS author_rating
FROM book_rating as br
LEFT JOIN book_authors as ba ON br.book_id = ba.book_id
ORDER BY author_rating DESC
LIMIT 1
'''

In [19]:
request(query_4_1)

,author,author_id,author_rating
0,J.K. Rowling/Mary GrandPré,236,4.283844


## Задача 5 -  Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

Здесь я также предлагаю 2 варианта решения задачи. 

**Вариант 1 - без использования оконных функций:**
1. Делаю подзапрос: из таблицы `ratings`, сгруппированной по имени пользователя `username`, выведем имена пользователей `username`, число оценок от которых `COUNT(rating_id)` более 50.
2. Из таблицы `reviews` выгрузим имена пользователей `username` и количество проведенных обзоров `COUNT(review_id) as counter`. Оставим только тех пользователей, чьи имена присутствуют в подзапросе 1 (количество оценок больше 50). 
3. Получившийся результат вынесем во временную таблицу `cte`.
4. Сделаем запрос из `cte`, в котором посчитаем среднее количество обзоров от выбранных пользователей, округленное до 2 знака. 

Результат: такие пользователи сделали в среднем 24.33 обзора. 

In [20]:
query_5 = '''
WITH cte
AS
(SELECT username, COUNT(review_id) as counter
FROM reviews AS r
WHERE username in (SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 50)
GROUP BY username)

SELECT ROUND(AVG(counter), 2)
FROM cte
'''

In [21]:
request(query_5)

,round
0,24.33


**Вариант 2 - с использованием оконных функций:**
1. Делаю подзапрос: из таблицы `ratings`, сгруппированной по имени пользователя `username`, выведем имена пользователей `username`, число оценок от которых `COUNT(rating_id)` более 50.
2. Сгруппируем таблицу `reviews` по именам пользователей `username` и оставим только тех пользователей, чьи имена присутствуют в подзапросе 1 (количество оценок больше 50). При помощи оконной функции `OVER()` посчитаем среднее количество обзоров `AVG(COUNT(review_id)` по выбранным пользователям, округленное до 2 знака `ROUND()`. Группировку `OVER (PARTITION BY )` здесь не ставлю, она не нужна. 
3. Т.к. оконная функция проставила одинаковый средний рейтинг по всем строкам, ограничим вывод одной строкой `LIMIT 1`. 

Результат: такие пользователи сделали в среднем 24.33 обзора. 

In [22]:
query_5_1 = '''
SELECT ROUND(AVG(COUNT(review_id)) OVER(), 2) AS total_avg
FROM reviews AS r
WHERE username in (SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 50)
GROUP BY username
LIMIT 1
'''

In [23]:
request(query_5_1)

,total_avg
0,24.33


# Вывод: 

В результате проведения работы была изучена база данных сервиса покупки книг по подписке. 

База данных представлена 5 таблицами: 
1. `books` - информация о выпущенных книгах,
2. `publishers` - информация о издателях,
3. `authors` - информация о авторах,
4. `ratings` - оценки книг от пользователей,
5. `reviews` - обзоры книг от пользователей. 

Каждая таблица имеет первичный ключ, связь между таблицами осуществялется при помощи внешних ключей. 

После проведения необходимых запросов была получена информация, которая помогут сформулировать ценностное предложение для нового продукта.
- **Задача №1** - после 01.01.2000 г. было выпущено **821 книг**.
- **Задача №2** - для каждой из книг посчитано количество обзоров и средняя оценка пользователей,
- **Задача №3** - издательство Penguin Books выпустило больше всех книг **(42 шт)** объемом более 50 страниц, 
- **Задача №4** - автор с самым высоким рейтингом книг - Джоан Роулинг **(рейтинг 4.28)**,
- **Задача №5** - среднее количество обзоров от пользователей, которые поставили больше 50 оценок - **24.33 шт.** 